In [4]:
# add dependencies
import pandas as pd
import os
import datetime

from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float, and_, Date

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [9]:
# collect the start and end dates of the trips
date_entry = input('Enter a start date for your trip in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
start_date = datetime.date(year, month, day)

date_entry = input('Enter an end date for your trip in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
end_date = datetime.date(year, month, day)

Enter a start date for your trip
Enter a date in YYYY-MM-DD format: 2011-12-01
Enter an end date for your trip:


In [ ]:
# Use create the engine and assign classes
engine = create_engine("sqlite:///hawaii.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

Station = Base.classes.Stations
Measurement = Base.classes.Measurements

session = Session(engine)

Precipitation Analysis

In [ ]:
# get the date 1 year ago
year_ago = start_date - datetime.timedelta(365)


In [ ]:
print (datetime.date.today() + datetime.timedelta(-365))